## XGBoost for multi-classification
- Giả sử bài toán có $C$ lớp.
- Sử dụng kỹ thuật One hot coding, mỗi out sẽ là 1 vector, có đúng 1 phần tử bằng 1, các phần tử còn lại bằng 0. Phần tử bằng 1 năm ở vị trí tương ứng với class đó, thể hiện rằng điểm dữ liệu đang xét rơi vào class này với xác suất bằng 1.
- Xác suất mô hình dự đoán điểm $i$ rơi vào lớp c là: $$\hat{P}_{ic} =  \frac{\exp{\hat{y}_{ic}}}{\sum_{j = 1}^C \exp{\hat{y}_{ij}}}$$

- Hàm mất mát: Sử dụng hàm Cross Entropy để đánh giá sự khác nhau giữa 2 phân phối xác suất:
    $$ H(\mathbf{p}, \mathbf{q}) = -\sum_{i = 1}^N p_i \log(q_i) $$

    + Áp dụng vào bài toán: Hàm mất mát tại điểm dữ liệu thứ $i$:
    $$
    \begin{align*}
    L(y_i, \hat{y}_i) &= -\sum_{j=1}^C y_{ij} \log(\hat{P}_{ij}) \\
      &= -\sum_{j=1}^C y_{ij} \log \left( \frac{\exp(\hat{y}_{ij})}{\sum_{c=1}^C \exp(\hat{y}_{ic})} \right) \\
      &= -\sum_{j=1}^C ( y_{ij} \hat{y}_{ij} - y_{ij} \log \sum_{c=1}^C \exp(\hat{y}_{ic})) \\ 
      &= -\sum_{j=1}^C y_{ij} \hat{y}_{ij} +  ( \sum_{j=1}^C y_{ij})  \log \sum_{c=1}^C \exp(\hat{y}_{ic}) \\
      &= -\sum_{j=1}^C y_{ij} \hat{y}_{ij} +  \log \sum_{c=1}^C \exp(\hat{y}_{ic}) \ \ \ \ \ \ \text{Do}  \sum_{j=1}^C y_{ij} = 1 \\
    \end{align*}
    $$


    + Tính gradient:
    $$
    \begin{align*}
    \frac{\partial L}{ \partial \hat{y}_{ij}} &= - y_{ij} + \frac{\exp{\hat{y}_{ij}}}{\sum_{c=1}^C \exp(\hat{y}_{ic})}  \\
                                              &= - y_{ij} + \hat{P}_{ij}
    \end{align*}
    $$

    Khi đó gradient:
    $$
    \begin{align*}
    g_i = \frac{\partial L}{ \partial \hat{y}_{i}} &= \left[\frac{\partial L}{ \partial \hat{y}_{i1}},..., \frac{\partial L}{ \partial \hat{y}_{iC}} \right]  \\
                                             &= \left[- y_{i1} + \hat{P}_{i1},..., - y_{ij} + \hat{P}_{ic} \right]
    \end{align*}
    $$

    + Tính Hessian: 
    $$
    \begin{align*}
    \frac{\partial^2 L}{ \partial^2 \hat{y}_{ij}} 
    &= \frac{\exp{(\hat{y}_{ij})} \sum_{c=1}^C \exp(\hat{y}_{ic}) - (\exp{(\hat{y}_{ij})})^2}{(\sum_{c=1}^C \exp(\hat{y}_{ic}))^2}  \\
    &= \frac{\exp{(\hat{y}_{ij})}}{\sum_{c=1}^C \exp(\hat{y}_{ic})} \left(1 - \frac{\exp{(\hat{y}_{ij})}}{\sum_{c=1}^C \exp(\hat{y}_{ic})} \right) \\
    &= \hat{P}_{ij} (1 - \hat{P}_{ij})
    \end{align*}
    $$

- Dự đoán: giá trị dự đoán dựa trên lớp có xác suất lớn nhất

In [19]:
import numpy as np
import pandas as pd
import math

class TreeBooster:
    def __init__(self, X, gradients, hessians, max_depth, min_child_weight, reg_lambda, gamma, idxs=None):
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        
        if idxs is None: 
            idxs = np.arange(len(gradients))
        self.X, self.gradients, self.hessians, self.idxs = X, gradients, hessians, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value_predict = -self.gradients[self.idxs].sum() / (self.hessians[self.idxs].sum() + self.reg_lambda)
        self.best_score_so_far = 0.
        self.best_feature_type = None
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)
        if self.is_leaf():
            return
        x = self.X[self.idxs, self.split_feature_idx]
        if self.best_feature_type == 'Categorical':
            left_idx = self.idxs[x == self.threshold]
            right_idx = self.idxs[x != self.threshold]
        else:
            left_idx = self.idxs[x <= self.threshold]
            right_idx = self.idxs[x > self.threshold]
        self.left = TreeBooster(self.X, self.gradients, self.hessians, self.max_depth - 1, self.min_child_weight, self.reg_lambda, self.gamma, left_idx)
        self.right = TreeBooster(self.X, self.gradients, self.hessians, self.max_depth - 1, self.min_child_weight, self.reg_lambda, self.gamma, right_idx)

    def is_leaf(self):
        return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X[self.idxs, feature_idx]
        g, h = self.gradients[self.idxs], self.hessians[self.idxs]
        sum_g, sum_h = g.sum(), h.sum()
        
        # Thuộc tính kiểu phân loại
        if x.dtype.kind in ['b', 'O']:
            #print('o')
            unique_elements = np.unique(x)
            ids_left = (x == unique_elements[0])
            ids_right = ~ids_left
            sum_g_left = g[ids_left].sum() 
            sum_h_left = h[ids_left].sum()
            sum_g_right = g[ids_right].sum()
            sum_h_right = h[ids_right].sum()

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                            + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                            - (sum_g**2 / (sum_h + self.reg_lambda))) - self.gamma

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold = unique_elements[0]
                self.best_feature_type = 'Categorical'
        
        #Thuộc tính kiểu số
        else:
            sort_idx = np.argsort(x)
            sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
            sum_g_right, sum_h_right = sum_g, sum_h
            sum_g_left, sum_h_left = 0., 0.

            for i in range(0, self.n - 1):
                g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
                sum_g_left += g_i
                sum_h_left += h_i
                sum_g_right -= g_i
                sum_h_right -= h_i
                if sum_h_left < self.min_child_weight or x_i == x_i_next:
                    continue
                if sum_h_right < self.min_child_weight:
                    break

                gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                            + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                            - (sum_g**2 / (sum_h + self.reg_lambda))) - self.gamma

                if gain > self.best_score_so_far:
                    self.split_feature_idx = feature_idx
                    self.best_score_so_far = gain
                    self.threshold = (x_i + x_i_next) / 2
                    self.best_feature_type = 'numeric'


    def predict(self, X):
        return np.array([self._predict_row(row) for row in X])

    def _predict_row(self, row):
        if self.is_leaf():
            return self.value_predict
        
        child = self.left if row[self.split_feature_idx] <= self.threshold else self.right
        return child._predict_row(row)


class XGBoostMultiClassifier:
    def __init__(self, learning_rate=0.1, max_depth=3, min_child_weight=1, gamma=0, reg_lambda=1, subsample=1.0, num_boost_round=100, random_seed=None):
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.gamma = gamma
        self.reg_lambda = reg_lambda
        self.subsample = subsample
        self.num_boost_round = num_boost_round
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y):
        self.num_classes = len(np.unique(y))
        self.boosters = [[] for _ in range(self.num_classes)]
        if isinstance(X, pd.DataFrame): 
            X = X.values
        if isinstance(y, pd.Series): 
            y = y.values

        y_one_hot = np.zeros((len(y), self.num_classes))

        for i in range(len(y)):
            y_one_hot[i, y[i]] = 1

        current_predictions = np.zeros((len(y), self.num_classes))
        
        for i in range(self.num_boost_round):
            for k in range(self.num_classes):
                # giảm thiểu tràn số
                probs = np.exp(current_predictions - np.max(current_predictions, axis=1, keepdims=True))
                probs /= probs.sum(axis=1, keepdims=True)
                #print(probs[1].sum())
                gradients = probs[:, k] - y_one_hot[:, k]
                hessians = probs[:, k] * (1 - probs[:, k])

                if self.subsample == 1.0:
                    sample_idxs = None
                else:
                    sample_idxs = self.rng.choice(len(y),
                                        size=math.floor(self.subsample * len(y)),
                                        replace=False)
                    
                booster = TreeBooster(X, gradients, hessians, self.max_depth, self.min_child_weight, self.reg_lambda, self.gamma, sample_idxs)
                current_predictions[:, k] += self.learning_rate * booster.predict(X)
                self.boosters[k].append(booster)

    def predict_proba(self, X):
        log_odds = np.array([np.sum([booster.predict(X) for booster in class_boosters], axis=0) for class_boosters in self.boosters])
        log_odds = log_odds.T 
        probabilities = np.exp(log_odds) / np.exp(log_odds).sum(axis=1, keepdims=True)
        return probabilities

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)

        

In [2]:
data = pd.read_csv('car_evaluation.csv')

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [3]:
X = pd.get_dummies(X)

In [4]:
from sklearn.model_selection import  train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
label = LabelEncoder()
y_train_1 = label.fit_transform(y_train)
y_test_1 = label.transform(y_test)

In [20]:
my_xgb = XGBoostMultiClassifier()
my_xgb.fit(X_train, y_train_1)
pred = my_xgb.predict(X_test)
accuracy_score(pred, y_test_1)

0.9682080924855492

### Sử dụng thư viện

In [18]:
import xgboost as xgb
xg_sklearn = xgb.XGBClassifier()
xg_sklearn.fit(X_train, y_train_1)
pred = xg_sklearn.predict(X_test)
accuracy_score(y_test_1, pred)

0.9739884393063584